In [7]:
import numpy as np
from packages.Utility.CNN.Callbacks import *
from packages.Utility.DatasetManager import DatasetReader
from packages.Utility.Gabor import GaborFilterBanks, ComplexGaborParams
from packages.Model.CFNet import CFNet1_1

# Inisialisasi / Persiapan

In [8]:
fold_index = 0
dataset_dir = 'dataset/bin/CF.32.B.30K'
trained_dir = 'trained/CF.32.B.30K CF1.1.3'

# Memuat Dataset

In [9]:
reader = DatasetReader(dataset_dir)

X_train, Y_train_onehot, Y_train_single = reader.getTrainData(fold_index)
X_test, Y_test_onehot, Y_test_single = reader.getTestData(fold_index)

# Inisiasi Callbacks

In [10]:
val_model = ValidateModel(X_train, Y_train_onehot)
save_model = SaveModel(val_model, base_dir = trained_dir, fold = fold_index + 1, min_acc = 0.1)
stop_train = StopTraining(val_model)
history_save = HistorySaver(val_model, base_dir = trained_dir, fold = fold_index + 1)

# Mendefinisikan Kernel Gabor

In [11]:
kernel1 = GaborFilterBanks(ComplexGaborParams(
    psi = (1, 10), gamma = (0.05, 0.75), sigma = (1, 10), lambd = (5, 10)
))
kernel2 = GaborFilterBanks(ComplexGaborParams(
    psi = (3, 13), gamma = (0.05, 0.75), sigma = (1, 10), lambd = (5, 10)
))
kernel3 = GaborFilterBanks(ComplexGaborParams(
    psi = (1, 13), gamma = (0.05, 0.75), sigma = (1, 10), lambd = (5, 10)
))
kernel4 = GaborFilterBanks(ComplexGaborParams(
    psi = (3, 13), gamma = (0.05, 0.75), sigma = (1, 10), lambd = (5, 10)
))
kernel5 = GaborFilterBanks(ComplexGaborParams(
    psi = (1, 13), gamma = (0.05, 0.75), sigma = (1, 10), lambd = (5, 10)
))
kernel6 = GaborFilterBanks(ComplexGaborParams(
    psi = (3, 13), gamma = (0.05, 0.75), sigma = (1, 10), lambd = (5, 10)
))

# Mendefinisikan Model

In [12]:
ars = CFNet1_1(kernels = {
    1: kernel1,
    2: kernel2,
    3: kernel3,
    4: kernel4,
    5: kernel5,
    6: kernel6
}, loss = 'categorical_crossentropy', optimizer = 'adadelta')
model = ars.model

In [ ]:
ars.save(f'{trained_dir}/Fold {fold_index + 1}/init.h5')
model.summary()

In [13]:
model.load_weights(f'trained/CF.32.B.30K CF1.1.3/Fold 1/acc 0.6120 - epoch 99.h5')

# Pelatihan

In [ ]:
ars.compile()
history = model.fit(X_train, Y_train_onehot, epochs = 300, verbose = 2, batch_size = 128, callbacks=[val_model, save_model, history_save, stop_train], initial_epoch = 100)